In [1]:
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

import pandas as pd

from dotenv import load_dotenv

from IPython.display import display, Markdown

In [2]:
# load_dotenv()

In [3]:
# llm = GoogleGenerativeAI(model="gemini-2.5-flash")
#
# response = llm.invoke("Summarize the gist of the quote in a paragraph in simple English: 'Not all those who wander are lost.'")

In [4]:
# display(Markdown(response))

In [5]:
quotes = pd.read_json("cleaned_quotes.json", lines=True)
quotes

,0,1,2,3,4,5,6,7,8,9,...,2984,2985,2986,2987,2988,2989,2990,2991,2992,2993
0,"All that is gold does not glitter,\nNot all th...",Not all those who wander are lost.\n,"I wish it need not have happened in my time,"" ...",I don't know half of you half as well as I sho...,All we have to decide is what to do with the t...,If more of us valued food and cheer and song a...,Never laugh at live dragons.\n,Faithless is he that says farewell when the ro...,"The world is indeed full of peril, and in it t...",Deserves it! I daresay he does. Many that live...,...,"Where did you go to, if I may ask?' said Thori...","In this hour, I do not believe that any darkne...",Oft hope is born when all is forlorn.\n,The wide world is all about you: you can fence...,I do not love the bright sword for its sharpne...,He that breaks a thing to find out what it is ...,"Home is behind, the world ahead,\nAnd there ar...","There is more in you of good than you know, ch...","Good Morning!"" said Bilbo, and he meant it. Th...","In sorrow we must go, but not in despair. Beho..."


In [6]:
len(quotes), len(quotes.columns), type(quotes)

(1, 2994, pandas.core.frame.DataFrame)

In [7]:
# transpose the columns
quotes = quotes.T
quotes

,0
0,"All that is gold does not glitter,\nNot all th..."
1,Not all those who wander are lost.\n
2,"I wish it need not have happened in my time,"" ..."
3,I don't know half of you half as well as I sho...
4,All we have to decide is what to do with the t...
...,...
2989,He that breaks a thing to find out what it is ...
2990,"Home is behind, the world ahead,\nAnd there ar..."
2991,"There is more in you of good than you know, ch..."
2992,"Good Morning!"" said Bilbo, and he meant it. Th..."


In [8]:
len(quotes), len(quotes.columns), type(quotes)

(2994, 1, pandas.core.frame.DataFrame)

In [9]:
prompt_template = PromptTemplate.from_template(
    "Summarize the following quote in simple English in a paragraph: '{quote}'")

In [35]:
# indexing in [column][row] format
prompt_template.invoke(quotes[0][0])

StringPromptValue(text="Summarize the following quote in simple English in a paragraph: 'All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.\n'")

In [11]:
prompt_template_list = []

In [12]:
for i in range(len(quotes)):
    prompt_template_list.extend(prompt_template.invoke(quotes[0][i]))

In [13]:
prompt_template_list[0][1]

"Summarize the following quote in simple English in a paragraph: 'All that is gold does not glitter,\nNot all those who wander are lost;\nThe old that is strong does not wither,\nDeep roots are not reached by the frost.\n\nFrom the ashes a fire shall be woken,\nA light from the shadows shall spring;\nRenewed shall be blade that was broken,\nThe crownless again shall be king.\n'"

### Confusion over iterating in `prompt_template_list`:

* [0][0]: 'text'
* [0][1]: 'The actual quote'
* [1][0]: 'type'
* [1][1]: 'StringPromptValue'

#### Trying a different approach: Appending the prompt individually to the quote using `quotes`

In [44]:
prompt_template_text = []

In [45]:
for i in range(len(quotes)):
    prompt_template_text.append(f"Summarize the following quote in simple English in a paragraph: {quotes[0][i]}")

In [48]:
len(prompt_template_text), type(prompt_template_text)

(2994, list)

In [55]:
type(quotes)

pandas.core.frame.DataFrame

In [58]:
quotes["prompt_template"] = prompt_template_text
quotes.rename(columns={0:"quote"}, inplace=True)
quotes

,quote,prompt_template
0,"All that is gold does not glitter,\nNot all th...",Summarize the following quote in simple Englis...
1,Not all those who wander are lost.\n,Summarize the following quote in simple Englis...
2,"I wish it need not have happened in my time,"" ...",Summarize the following quote in simple Englis...
3,I don't know half of you half as well as I sho...,Summarize the following quote in simple Englis...
4,All we have to decide is what to do with the t...,Summarize the following quote in simple Englis...
...,...,...
2989,He that breaks a thing to find out what it is ...,Summarize the following quote in simple Englis...
2990,"Home is behind, the world ahead,\nAnd there ar...",Summarize the following quote in simple Englis...
2991,"There is more in you of good than you know, ch...",Summarize the following quote in simple Englis...
2992,"Good Morning!"" said Bilbo, and he meant it. Th...",Summarize the following quote in simple Englis...


In [68]:
quotes.to_json("quotes_with_prompt.json", orient="index")